Импорт зависимостей

In [7]:
import sys
import numpy as np
from scipy import ndimage
import matplotlib.pyplot as plt
from PIL import Image

Оператор Робертса

In [8]:
robertsCrossV = np.array([[ 0, 0, 0 ], [ 0, 1, 0 ], [ 0, 0,-1 ]])
robertsCrossH = np.array([[ 0, 0, 0 ], [ 0, 0, 1 ], [ 0,-1, 0 ]])

def robertsCross(imageData):
    vertical = ndimage.convolve(imageData, robertsCrossV )
    horizontal = ndimage.convolve(imageData, robertsCrossH )
    return np.sqrt( np.square(horizontal) + np.square(vertical))

Объявление функций

In [9]:
def ShowImage(img, name):
    fig, axes = plt.subplots(1)
    axes.imshow(img, cmap="gray")
    axes.set_title(name)
    axes.axis("off")    
    fig.set_figwidth(10)    
    fig.set_figheight(10)  
    plt.show()

def PrintHist(imageArray, text):
    x = [i for i in range(0,256)]
    y = np.zeros(256, dtype = int)
    y1 = np.zeros(256, dtype = int)
    y2 = np.zeros(256, dtype = int)
    y3 = np.zeros(256, dtype = int)

    if len(imageArray.shape)>2:
      for i in imageArray[:,:,0].flat:
        y[i] += 1
      for i in imageArray[:,:,1].flat:
        y1[i] += 1
      for i in imageArray[:,:,2].flat:
        y2[i] += 1
      for i in imageArray.flat:
        y3[i] += 1
    else:
      for i in imageArray.flat:
        y3[i] += 1

    return IsLight(y3, imageArray)

def IsLight(lightArray, imgArray):
  lightArray = lightArray / imgArray.size
  treshHold = (np.max(lightArray) * 10)/100
  newLightArray = lightArray > treshHold
  newLightArraySum = newLightArray.sum()

  if newLightArraySum > 7:
    return "- Тоновое"
  elif newLightArraySum <= 2 and newLightArraySum != 0:
    return "- Штриховое"
  else:
    return "- Многоградационное"

def IsGreyScale(img):
  imgRGB = img.convert('RGB')
  w,h = imgRGB.size

  for i in range(w):
    for j in range(h):
      r,g,b = imgRGB.getpixel((i,j))
      if r != g != b: return False

  return True

Основная программа

In [13]:
pictures = ["1","2","3","4","5","6","7","8","9","10"]

for picture in pictures:

    image = Image.open(f'originals/{picture}.jpg')
    imgArr = np.asarray(image)

    imageData = np.asarray(image.convert("L"), dtype="int32")
    middle = int(np.median(imageData)) #Средняя яркость пикселей
    lightness=""
    light_level=0
    if(middle in range(130,255)):
        lightness="светлое"
        light_level=0.78
    elif(middle in range(100,130)):
        lightness="среднее"
        light_level=0.46
    elif(middle in range(0,100)):
        lightness="тёмное"
        light_level=0.11

    borderMask = imageData > middle*light_level
    imageData[borderMask] = 255
    imageData[np.invert(borderMask)] = 0

    robertsImageData = robertsCross(imageData)
    tonesPixelsCount = np.count_nonzero(robertsImageData < 128)
    imagePixelsCount = imageData.shape[0] * imageData.shape[1]
    tonesPercent = tonesPixelsCount / imagePixelsCount
    
    print(f'\nИзображение {picture}:')
    color=False
    print(f"- Яркость: {lightness}")
    print(f"- Площадь, занятая равномерными тонами: {tonesPercent}")
    if tonesPercent < 0.94:
        print(f"- Изображение с большим количеством сюжетных деталей")
    elif tonesPercent < 0.96:
        print(f"- Среднее содержание контуров")
    else:
        print(f"- Преобладают равномерные тона")
    if IsGreyScale(image) == True:
        print("- Черно-белое")
    else:
        print("- Цветное")
        color=True

    gradatsii=PrintHist(imgArr, "sas")
    print(PrintHist(imgArr, "sas"))
    if(color==True and gradatsii=="- Многоградационное"):
        print("Рекомендуется: цветоделение")
    elif(gradatsii=="-. Тоновое" or gradatsii=="- Многоградационное"):
        print("Рекомендуется: растрирование")
    elif(color==True):
        print("Рекомендуется: цветоделение")
    else:
        print("Рекомендации отсутствуют")


Изображение 1:
- Яркость: светлое
- Площадь, занятая равномерными тонами: 0.8659991634424343
- Изображение с большим количеством сюжетных деталей
- Черно-белое
- Многоградационное
Рекомендуется: растрирование

Изображение 2:
- Яркость: тёмное
- Площадь, занятая равномерными тонами: 0.9503408382011002
- Среднее содержание контуров
- Черно-белое
- Штриховое
Рекомендации отсутствуют

Изображение 3:
- Яркость: светлое
- Площадь, занятая равномерными тонами: 0.9802667301318666
- Преобладают равномерные тона
- Цветное
- Тоновое
Рекомендуется: цветоделение

Изображение 4:
- Яркость: светлое
- Площадь, занятая равномерными тонами: 0.9792874268387278
- Преобладают равномерные тона
- Черно-белое
- Тоновое
Рекомендации отсутствуют

Изображение 5:
- Яркость: светлое
- Площадь, занятая равномерными тонами: 0.8966560748212579
- Изображение с большим количеством сюжетных деталей
- Черно-белое
- Тоновое
Рекомендации отсутствуют

Изображение 6:
- Яркость: тёмное
- Площадь, занятая равномерными тонами: